In [1]:
#Array packages
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nc4

from scipy.stats import kendalltau

#plots
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import geopandas as gpd
import scienceplots
import cartopy.feature as cfeature


#Progress meter
from dask.diagnostics import ProgressBar
from tqdm import tqdm

# Directories
import os
import glob
import dask
#import h5netcdf
import scipy

import os
os.chdir(r"G:\OneDrive - IIT Delhi\3. IIT DELHI\2. Research\2_Papers\1_Streamflow Trend analysis")
print(os.getcwd())

G:\OneDrive - IIT Delhi\3. IIT DELHI\2. Research\2_Papers\1_Streamflow Trend analysis


## 1 Loading Data

In [2]:
gauge_info=pd.read_csv('3_Data/Data_p/2_Station/gauge_info_p.csv',index_col=0)
gauge_info1=gauge_info.iloc[:,1:5]
gauge_sf=xr.open_dataset(f'3_Data/Data_p/2_Station/Gauge_streamflow.nc')
ILDAS_sf=xr.open_mfdataset(f'3_Data/Data_p/1_ILDAS/ILDAS_station_streamflow.nc')
south_asia=gpd.read_file(r'3_Data\Data_p\3_Shapefiles\south_asia_p.shp')

In [5]:
gauge_sf_m=gauge_sf.resample(time='M').mean()
ILDAS_sf_m=ILDAS_sf.resample(time='M').mean()
tim=pd.date_range(start="1980-01", end="2020-12", freq="M")
gauge_info1


stations=gauge_sf_m['station'].values
tim=pd.date_range(start="1980-01", end="2020-12", freq="M").to_numpy(dtype='datetime64[ns]')

In [80]:

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.serif"] = "Times New Roman"



df=pd.DataFrame(index=tim,columns=['ILDAS','Gauge'])





for i,stn in tqdm(enumerate(stations)):

    df['Gauge']=gauge_sf_m['streamflow'].sel(station=stn,time=tim)
    df['ILDAS']=ILDAS_sf_m['streamflow'].sel(station=stn,time=tim)

    fig = plt.figure(figsize=(10, 12))
    gs = fig.add_gridspec(2, 1, height_ratios=[2.7, 1])  # Controls the height ratio of subplots

    # Plot 1 - Timeseries
    ax1 = fig.add_subplot(gs[1])  
    ax1.plot(tim, df['Gauge'], color='blue', label='Gauge', alpha=1, linewidth=1.3)
    ax1.plot(tim, df['ILDAS'], color='red', label='ILDAS', alpha=1, linewidth=1,linestyle='--')


    [x.set_linewidth(1.4) for x in ax1.spines.values()]

    ax1.set_xlabel('Time',fontsize=14,labelpad=7,weight='bold')
    ax1.set_ylabel('Streamflow',fontsize=14,labelpad=7,weight='bold')
    ax1.tick_params(which='major', length=0)
    ax1.tick_params(which='minor', length=0)
    #ax1.set_ylim(22,33)

    lgnd=plt.legend(loc='center', ncol=2,fontsize=13, bbox_to_anchor=(0.79, 0.92),frameon=True, borderpad=0.32, edgecolor='black')
    frame = lgnd.get_frame()
    frame.set_linewidth(0.8)

    # # Plot 2 - Map
    ax0 = fig.add_subplot(gs[0], projection=ccrs.PlateCarree())  # Set the projection for the map
    south_asia.plot(ax=ax0, color='none', edgecolor='Black', linewidth=0.8, alpha=1)
    [x.set_linewidth(1.8) for x in ax0.spines.values()] 
    #ax0.coastlines(resolution='110m', linewidth=1)

    ax0.scatter(gauge_info1['Longitude'],gauge_info1['Latitude'],s=10)
    ax0.scatter(gauge_info1[gauge_info1['Station']==stn]['Longitude'],gauge_info1[gauge_info1['Station']==stn]['Latitude'],color='red',s=80)


    #ax0.set_title(stations[0].values)

    ax0.text(0.72, 0.85, stn, fontsize=18, color='black',weight='bold',ha='right', va='top', transform=ax0.transAxes)

    # Adjust the layout to avoid overlap
    fig.tight_layout(pad=2.0)  # Increase padding if needed


    plt.savefig(f'2_Streamflow_Analysis/2_basic_statistics/Outputs/Timeseries plots/{i+1}_{stn}.jpeg')
    plt.close(fig)

241it [03:08,  1.28it/s]


In [ ]:
#Create GIF
import imageio
images = [imageio.imread(f'2_Streamflow_Analysis/2_basic_statistics/Outputs/Timeseries plots/{i+1}_{stn}.jpeg') for i,stn in enumerate(stations)]
imageio.mimsave(f'2_Streamflow_Analysis/2_basic_statistics/Outputs/streamflow_timeseries_GIF.gif', images,duration=800,loop=0,dpi=300)  
#df=pd.DataFrame(index=ILDAS_ind['time'].values,columns=['Gauge','ILDAS'])


C:\Users\2024CEZ8029\AppData\Local\Temp\ipykernel_4688\3443012196.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(f'2_Streamflow_Analysis/2_basic_statistics/Outputs/Timeseries plots/{i+1}_{stn}.jpeg') for i,stn in enumerate(stations)]


: 

###